In [ ]:
# Step 1

import pandas as pd

# Read the Excel file with LCMS analysis. Insert file pathway. 
data = pd.read_excel('insert file pathway')

# Keep only columns 'Accession', 'Description', '# Peptides', 'Area'
data = data[['Accession', 'Description', '# Peptides', 'Area']]

# Sort the DataFrame by values in column '# Peptides'
data = data.sort_values(by='# Peptides')

# Filter rows based on values in column '# Peptides'
data = data[data['# Peptides'] >= 3]

# Print DataFrame
data

In [ ]:
# Step 2

# Filter rows containing "Large ribosomal subunit" and "Escherichia coli"
mask_large = data['Description'].str.contains('Escherichia coli', case=False) & data['Description'].str.contains('Large ribosomal subunit', case=False)
selected_rows_large = data[mask_large]

# Filter rows containing "Small ribosomal subunit" and "Escherichia coli"
mask_small = data['Description'].str.contains('Escherichia coli', case=False) & data['Description'].str.contains('Small ribosomal subunit', case=False)
selected_rows_small = data[mask_small]

# Filter rows containing "30S ribosomal protein" and "Escherichia coli"
mask_30s = data['Description'].str.contains('Escherichia coli', case=False) & data['Description'].str.contains('30S ribosomal protein', case=False)
selected_rows_30s = data[mask_30s]

# Filter rows containing "50S ribosomal protein" and "Escherichia coli"
mask_50s = data['Description'].str.contains('Escherichia coli', case=False) & data['Description'].str.contains('50S ribosomal protein', case=False)
selected_rows_50s = data[mask_50s]

# Remove selected rows from the original DataFrame
data = data[~mask_large & ~mask_small & ~mask_30s & ~mask_50s]

# Concatenate selected rows at the top of the DataFrame
data = pd.concat([selected_rows_large, selected_rows_small, selected_rows_30s, selected_rows_50s, data])

# Now, 'data' contains rows with specified text combinations at the top


# To print the entire DataFrame and check that everuthing worked
pd.set_option('display.max_rows', None)  # Set to None to display all rows
pd.set_option('display.max_columns', None)  # Set to None to display all columns

# Calculating Ribosomal Average
for _ in range(2):
    # Concatenate all selected rows into one DataFrame
    selected_rows = pd.concat([selected_rows_large, selected_rows_small, selected_rows_30s, selected_rows_50s]).reset_index(drop=True)

    # Calculate the average 'Area' for all selected rows
    average_area_all = selected_rows['Area'].mean()

    # Create a row for ribosomal average
    ribosomal_average_row = pd.DataFrame({'Description': 'Ribosomal average', 'Area': average_area_all}, index=[0])

    # Find the index to insert the new row below the selected rows in the main DataFrame
    index_to_insert = data.index[data['Description'].isin(selected_rows['Description'])][-1] + 1

    # Insert the row with the calculated average into the main DataFrame below the selected rows
    data = pd.concat([data.iloc[:index_to_insert], ribosomal_average_row, data.iloc[index_to_insert:]]).reset_index(drop=True)

# Now, 'data' contains the main DataFrame with a new row containing the calculated average for 'Area' under the selected rows

data

# Scroll down for Step 3. 

In [ ]:
# Step 3

# Calculate the Ribosomal average
ribosomal_average = data.loc[data['Description'] == 'Ribosomal average', 'Area'].iloc[0]

# Create a new column 'Result' with values calculated by dividing 'Area' by 'Ribosomal average'
data['Result'] = data['Area'] / ribosomal_average

# Replace 'value_to_find' with the specific value in the 'Accession' column. Accession might differ in each sample.
value_to_find = 'A0A0X9U5K8'

# Find the row with the specified 'Accession' value and print the corresponding value from the 'Result' column
result_value = data.loc[data['Accession'] == value_to_find, 'Result'].values[0]
print("Concentration of the target protein = ", result_value)